In [1]:
! pip3 install -qU langchain-upstage python-dotenv langchain_core markdown langchain_community langchain_text_splitters langchain_chroma tavily-python


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext dotenv
%dotenv

In [3]:
import markdown

# Convert markdown file to HTML
def convert_markdown_to_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    html = markdown.markdown(content)
    return html

file_path = 'apply-home.md'
markdown_string = ''
with open(file_path, 'r', encoding='utf-8') as file:
    markdown_string = file.read()
print(len(markdown_string))

109097


In [4]:
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.MARKDOWN
)
splits = text_splitter.split_text(markdown_string)

In [5]:
import re

def split_markdown_by_h2(content):
    # Use regex to split the content by `##` titles
    sections = re.split(r'(## .+)', content)
    # Initialize an empty list to hold the final sections
    split_content = []
    
    # Combine the titles with their corresponding content
    for i in range(1, len(sections), 2):
        section_title = sections[i]
        section_content = sections[i + 1] if (i + 1) < len(sections) else ''
        split_content.append(section_title + section_content)
    
    return split_content

section = split_markdown_by_h2(markdown_string)
print(section[0][:100])

## 청약주택
#


In [6]:
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings

split_docs = [Document(page_content=text) for text in splits]

# 3. Embed & indexing
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

In [7]:
retriever = vectorstore.as_retriever()
result_docs = retriever.invoke("청약 자격")
print(len(result_docs))
print(result_docs[0].page_content[:100])

4
- 대상주택
    - 전용면적 85m² 이하의 분양주택 (국민주택, 민영주택)
    - 한부모가족, 국군포로, 위안부피해자, 철거주택세입자 기관추천의 경우에는 공공임대주택
- 


In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


# RAG or Search?
def is_in(question, context):
    is_in_conetxt = """As a helpful assistant, 
please use your best judgment to determine if the answer to the question is within the given context. 
If the answer is present in the context, please respond with "yes". 
If not, please respond with "no". 
Only provide "yes" or "no" and avoid including any additional information. 
Please do your best. Here is the question and the context:
---
CONTEXT: {context}
---
QUESTION: {question}
---
OUTPUT (yes or no):"""

    is_in_prompt = PromptTemplate.from_template(is_in_conetxt)
    chain = is_in_prompt | ChatUpstage() | StrOutputParser()

    response = chain.invoke({"context": context, "question": question})
    print(response)
    return response.lower().startswith("yes")

In [10]:
import os
from tavily import TavilyClient
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()
prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)

def get_smart_rag(question, context):
    if not is_in(question, context):
        print("Searching in tavily")
        tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
        context = tavily.search(query=question)

    return context

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AIMessage, HumanMessage

query = '우리가 청약할 수 있는 분양공고인지 알고 싶어'
context = get_smart_rag(question=query, context=result_docs)

# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a helpful assistant. Please help user.
         ---
         Context: {context}
         """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
chain = chat_with_history_prompt | ChatUpstage(streaming=True) | StrOutputParser()

no
Searching in tavily


In [15]:
def chat(message, history):
    history_langchain_format = []
    if history is not None:
        for human, ai in history:
            history_langchain_format.append(HumanMessage(content=human))
            history_langchain_format.append(AIMessage(content=ai))

    generator = chain.stream({"message": message, "history": history_langchain_format, "context": context})

    assistant = ""
    for gen in generator:
        assistant += gen
        yield assistant

In [17]:
import gradio as gr

# Create Gradio interface
iface = gr.ChatInterface(
    fn=chat,
    inputs="text",
    outputs="text",
    title="Chat with LLM using LangChain",
    description="Type your question and interact with the AI."
)

# Launch the app
iface.launch(share=True)

TypeError: ChatInterface.__init__() got an unexpected keyword argument 'inputs'